In [ ]:
!pip install sentence-transformers faiss-cpu PyMuPDF langchain


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving AI Training Document.pdf to AI Training Document.pdf


In [ ]:
import fitz  # PyMuPDF

doc = fitz.open("AI Training Document.pdf")
text = ""
for page in doc:
    text += page.get_text()

print(text[:1000])  # pehle 1000 characters check karne ke liye


User Agreement 
1. Introduction 
This User Agreement, the Mobile Application Terms of Use, and all policies and additional terms 
posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms 
on which eBay offers you access to and use of our Services. You can find an overview of our policies 
here. The Mobile Application Terms of Use, all policies, and additional terms posted on and in our 
Services are incorporated into this User Agreement. You agree to comply with all terms of this User 
Agreement when accessing or using our Services. 
The entity you are contracting with is: eBay Inc., 2025 Hamilton Ave., San Jose, CA 95125, if you 
reside in the United States; eBay (UK) Limited, 1 More London Place, London, SE1 2AF, United 
Kingdom, if you reside in the United Kingdom; eBay GmbH, Albert-Einstein-Ring 2-6, 14532 
Kleinmachnow, Germany, if you reside in the European Union; eBay Canada Limited, 240 Richmond 
Street West, 2nd Floor Suite 02-1

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
)

chunks = text_splitter.create_documents([text])
print(f"Total chunks: {len(chunks)}")
print("First chunk:\n", chunks[0].page_content)


Total chunks: 270
First chunk:
 User Agreement 
1. Introduction 
This User Agreement, the Mobile Application Terms of Use, and all policies and additional terms 
posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
print("✅ Model loaded!")

texts = [chunk.page_content for chunk in chunks]
embeddings = model.encode(texts)
print(f"Embeddings shape: {embeddings.shape}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model loaded!
Embeddings shape: (270, 384)


In [ ]:
import faiss
import numpy as np

# Convert embeddings to numpy array
embedding_matrix = np.array(embeddings).astype("float32")

# FAISS index initialize
index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # 384 dim
index.add(embedding_matrix)

print("✅ FAISS index created with", index.ntotal, "vectors.")


✅ FAISS index created with 270 vectors.


In [ ]:
def search_documents(query, top_k=3):
    # Query embedding
    query_embedding = model.encode([query]).astype("float32")

    # FAISS search
    distances, indices = index.search(query_embedding, top_k)

    print("\n🔍 Top Matches:")
    for i in indices[0]:
        print("\n-- Chunk", i, "--")
        print(chunks[i].page_content)


In [ ]:
search_documents("What is this user agreement about?")



🔍 Top Matches:

-- Chunk 255 --
20. General 
Except as otherwise provided in this User Agreement, if any provision of this User Agreement is held 
to be invalid, void or for any reason unenforceable, such provision shall be struck out and shall not

-- Chunk 6 --
Agreement. 
Read this User Agreement carefully as it contains provisions that govern how claims you and we have 
against each other are resolved (see "Disclaimer of Warranties; Limitation of Liability" and "Legal

-- Chunk 0 --
User Agreement 
1. Introduction 
This User Agreement, the Mobile Application Terms of Use, and all policies and additional terms 
posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms


In [ ]:
def generate_response(query, top_k=3):
    # Query embedding
    query_embedding = model.encode([query]).astype("float32")

    # Search
    distances, indices = index.search(query_embedding, top_k)

    # Collect top chunk texts
    context = "\n\n".join([chunks[i].page_content for i in indices[0]])

    # Simulated response (basic version without LLM)
    print("💬 Response:\n")
    print(f"Based on the document, here's what I found for your question:\n\n{context}")


In [ ]:
generate_response("What is this user agreement about?")


💬 Response:

Based on the document, here's what I found for your question:

20. General 
Except as otherwise provided in this User Agreement, if any provision of this User Agreement is held 
to be invalid, void or for any reason unenforceable, such provision shall be struck out and shall not

Agreement. 
Read this User Agreement carefully as it contains provisions that govern how claims you and we have 
against each other are resolved (see "Disclaimer of Warranties; Limitation of Liability" and "Legal

User Agreement 
1. Introduction 
This User Agreement, the Mobile Application Terms of Use, and all policies and additional terms 
posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms
